# Logistic Regression from Scratch

## Logistic Regression
Reference [link](https://github.com/UCB-w261/main/blob/main/HelpfulResources/logistic_regression/Logistic%20regression.ipynb)

Logistic regression is an algorithm that could be used when the dependant variable \\(y\\) has a dichotomous outcome of a negative class or a postive class, \\(y \in\\) {0,1}

Some examples of the classification problem where logistic regression can be used.
* Email classification : Spam or not spam
* Credit card transactions: Fraud or not fraud
* Airline flights: Delayed or not delayed

### Notations

Below are the notations that are used in the explanation of logistic regression<br/>
\\(n\\) → number of features<br/>
\\(m\\) → number of training examples<br/>
\\(X\\) → input data matrix of shape (m x n)<br/>
\\(y\\) → true/ target value (can be 0 or 1 only)<br/>
\\(x(i), y(i)\\) → ith training example<br/>
\\(w\\) → weights (parameters) of shape (n x 1)<br/>
\\(b\\) → bias (parameter), a real number that can be broadcasted<br/>
\\({y}^{-}\\) → hypothesis \\({h}_{\theta}(X)\\) (outputs values between 0 and 1)<br/>

In logistic regression \\(0 \le {h}_{\theta}(X) \le 1 \\)

### Hypothesis

The hypothesis is \\({h}_{\theta}(X)\\) is the estimated probability that \\(y = 1\\) given the set of features \\(X\\) and parameterized by \\(\theta\\).<br/> 
\\(P(y=1 | X;\theta)\\)

In logistic regression, the hypothesis \\({h}_{\theta}(X) \\) is a Sigmoid function \\(g\\) where \\(g(z) = \frac{1}{1+{e}^{-z}}\\) and \\(z\\) = \\({\theta}^{T}X\\).

Hence, the hypothesis can be re-written as \\({h}_{\theta}(X) = \frac{1}{1+{e}^{-{\theta}^{T}X}}\\)

The probability that \\(y = 0\\) can be calculated by \\(1 - P(y=1 | X;\theta)\\), since the probability of both classes will add up to \\(1\\).

###Cost Function
For gradient descent to work, the cost function for logistic regression has to be a convex function. <br/>
Cost function is defined as

\\(J(\theta) = \frac{1}{m} {\Sigma}_{i=1}^{m} Cost(h_\theta(X(i)),y(i))\\)

where \\(y =\\) 0 or 1 and \\(Cost(h_\theta (X),y) = -y log(h_\theta (X)) - (1-y) log(1 - h_\theta(X))\\)

rewriting the cost function

\\(J(\theta) = - \frac{1}{m} [ {\Sigma}_{i=1}^{m} y(i) log(h_\theta (X(i))) + (1-y(i)) log(1 - h_\theta(X(i))) ]\\)

Here, \\(h_\theta(X)\\) is the Sigmoid function \\(\frac{1}{1+{e}^{-{\theta}^{T}X}}\\)

###Gradient Descent

We can use the gradient descent algorithm to fit the parameters \\(\theta\\) that minimizes the cost function \\(J(\theta)\\).

Gradient update step uses \\(\theta_j := \theta_j - \alpha \frac{d}{d\theta_j} J(\theta)\\) where \\(\alpha\\) is the learning rate and all \\(\theta_j\\) are updated simultaneously.

\\(\frac{d}{d\theta_j} J(\theta) = \frac{1}{m} {\Sigma}_{i=1}^{m} (h_\theta(X(i))- y(i)) X_j(i)\\)

The gradient descent step is re-written as

\\(\theta_j := \theta_j - \alpha \frac{1}{m} {\Sigma}_{i=1}^{m} (h_\theta(X(i))- y(i)) X_j(i)\\), all \\(\theta_j\\) are updated simultaneously.

Here, \\(h_\theta(X)\\) is the Sigmoid function \\(\frac{1}{1+{e}^{-{\theta}^{T}X}}\\)

###Make Predictions

To fit the parameters \\(\theta\\) we need to find the \\(\theta\\) such that it minimizes the cost function \\(J(\theta)\\). Using this set of \\(\theta\\) we can make predictions.

For predictions on a new feature set \\(X\\), calculate \\({h}_{\theta}(X) = \frac{1}{1+{e}^{-{\theta}^{T}X}}\\), which will give the probalility that \\(y=1\\).

We can use a threshold value for the probability to predict 0, if less than threshold, and 1, if greater than or equal to threshold probability.

### Data pre-processing

In [ ]:
# Load Libraries
from pyspark.sql.functions import col
from pyspark.sql import types
from pyspark.sql.functions import *
from pyspark.sql.types import LongType, StringType, IntegerType, DoubleType
from pyspark.sql.functions import udf
from pyspark.sql.functions import percent_rank
from pyspark.sql import Window
from pyspark import StorageLevel
from pyspark.sql import Row
from pyspark.sql.types import *

from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, StandardScaler, Imputer, MinMaxScaler, PCA
from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.classification import LogisticRegression, LogisticRegressionModel
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
import mlflow
import mlflow.sklearn

import re
import os
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import warnings

##### Set up blob storage

In [ ]:
# Parameters for SAS Token. Required for establishing connection to blob storage.

blob_container = "cont" # The name of your container created in https://portal.azure.com
storage_account = "acc" # The name of your Storage account created in https://portal.azure.com
secret_scope = "scope" # The name of the scope created in your local computer using the Databricks CLI
secret_key = "key" # The name of the secret key created in your local computer using the Databricks CLI 
blob_url = f"wasbs://{blob_container}@{storage_account}.blob.core.windows.net"
mount_path = "/mnt/path"

In [ ]:
spark.conf.set(
  f"fs.azure.sas.{blob_container}.{storage_account}.blob.core.windows.net",
  dbutils.secrets.get(scope = secret_scope, key = secret_key)
)

####Load data from blob storage

In [ ]:
# SMALL DATASET (6M)
joined_data_6M = spark.read.parquet(f"{blob_url}/joined_data")
# LARGE DATASET (2015-2018)
joined_data_traintest = spark.read.parquet(f'{blob_url}/joined_data_traintest')
# LARGE DATASET (2019)
joined_data_holdout = spark.read.parquet(f'{blob_url}/joined_data_holdout')

#### Features
- Columns to keep

In [ ]:
keep = ['ELEVATION_ORG', 'ELEVATION_DEST', 'FL_DATE', 'YEAR', 'QUARTER', 'MONTH', 'DAY_OF_MONTH', 'DAY_OF_WEEK', 'OP_UNIQUE_CARRIER', 'ORIGIN', 'ORIGIN_STATE_FIPS', 'DEST', 'DEST_STATE_FIPS', 'DEP_DEL15', 'DEP_TIME_BLK', 'ARR_TIME_BLK', 'DISTANCE_GROUP', 'BAD_TRAVEL_DAY', 'SLP_ORG', 'VIS_ORG', 'TMP_ORG', 'DEW_ORG', 'CIG_ORG', 'WND_ORG', 'GUST_ORG', 'TCB_ORG', 'OBSC_ORG', 'PREC_ORG', 'SNW_ORG', 'FZFG_ORG', 'FZDZ_ORG', 'FZRA_ORG', 'TSRA_ORG', 'HSN_ORG', 'LSN_ORG', 'BSN_ORG', 'VV_ORG', 'TS_ORG', 'HAIL_ORG', 'FOG_ORG', 'VA_ORG', 'BR_ORG', 'SQ_ORG', 'FC_ORG',  'SLP_DEST', 'VIS_DEST', 'TMP_DEST', 'DEW_DEST', 'CIG_DEST', 'WND_DEST', 'GUST_DEST', 'TCB_DEST', 'OBSC_DEST', 'PREC_DEST', 'SNW_DEST', 'FZFG_DEST', 'FZDZ_DEST', 'FZRA_DEST', 'TSRA_DEST', 'HSN_DEST', 'LSN_DEST', 'BSN_DEST', 'VV_DEST', 'TS_DEST', 'HAIL_DEST', 'FOG_DEST', 'VA_DEST', 'BR_DEST', 'SQ_DEST', 'FC_DEST', 'DEP_TIME_SEGMENT', 'p_rank']

- Columns to drop

In [ ]:
drop = ['DISTANCE', 'CRS_ARR_TIME', 'CRS_DEP_TIME', 'OP_CARRIER_AIRLINE_ID', 'OP_CARRIER_FL_NUM', 'DEP_TIMESTAMP', 'DEP_TIMESTAMP_UTC', 'DEP_TIMESTAMP_UTC_MINUS2', 'WX_STATION_ORIG', 'WX_STATION_DEST', 'CALL_SIGN_ORG', 'WX_TIMESTAMP_UTC_START_ORG', 'WX_TIMESTAMP_UTC_END_ORG', 'CALL_SIGN_DEST', 'WX_TIMESTAMP_UTC_START_DEST', 'WX_TIMESTAMP_UTC_END_DEST', 'DEST_AIRPORT_ID', 'DEST_CITY_MARKET_ID', 'ORIGIN_AIRPORT_ID', 'ORIGIN_CITY_MARKET_ID', 'join_year', 'year_start']

In [ ]:
#drop = ['DISTANCE', 'CRS_ARR_TIME', 'CRS_DEP_TIME', 'OP_CARRIER_AIRLINE_ID', 'OP_CARRIER_FL_NUM', 'DEP_TIMESTAMP', 'DEP_TIMESTAMP_UTC', 'DEP_TIMESTAMP_UTC_MINUS2', 'WX_STATION_ORIG', 'WX_STATION_DEST', 'CALL_SIGN_ORG', 'WX_TIMESTAMP_UTC_START_ORG', 'WX_TIMESTAMP_UTC_END_ORG', 'CALL_SIGN_DEST', 'WX_TIMESTAMP_UTC_START_DEST', 'WX_TIMESTAMP_UTC_END_DEST', 'DEST_AIRPORT_ID', 'DEST_CITY_MARKET_ID', 'ORIGIN_AIRPORT_ID', 'ORIGIN_CITY_MARKET_ID', 'join_year', 'year_start','FL_DATE', 'YEAR', 'QUARTER', 'DEST_STATE_FIPS', 'DISTANCE_GROUP', 'BAD_TRAVEL_DAY','WND_ORG', 'GUST_ORG', 'TCB_ORG', 'OBSC_ORG', 'SNW_ORG', 'FZFG_ORG', 'FZDZ_ORG', 'FZRA_ORG', 'TSRA_ORG', 'HSN_ORG', 'LSN_ORG', 'BSN_ORG', 'VV_ORG', 'TS_ORG', 'HAIL_ORG', 'FOG_ORG', 'VA_ORG', 'BR_ORG', 'SQ_ORG', 'FC_ORG',  'SLP_DEST', 'VIS_DEST', 'TMP_DEST', 'DEW_DEST','WND_DEST', 'GUST_DEST', 'TCB_DEST', 'OBSC_DEST','SNW_DEST', 'FZFG_DEST', 'FZDZ_DEST', 'FZRA_DEST', 'TSRA_DEST', 'HSN_DEST', 'LSN_DEST', 'BSN_DEST', 'VV_DEST', 'TS_DEST', 'HAIL_DEST', 'FOG_DEST', 'VA_DEST', 'BR_DEST', 'SQ_DEST', 'FC_DEST', 'DEP_TIME_SEGMENT']

- Columns that need one-hot-encoding (and string indexing)

In [ ]:
need_ohe = ['YEAR','QUARTER', 'MONTH', 'DAY_OF_MONTH', 'DAY_OF_WEEK', 'OP_UNIQUE_CARRIER', 'ORIGIN', 'ORIGIN_STATE_FIPS', 'DEST', 'DEST_STATE_FIPS', 'DEP_TIME_BLK', 'ARR_TIME_BLK', 'DISTANCE_GROUP', 'BAD_TRAVEL_DAY', 'DEP_TIME_SEGMENT']

In [ ]:
#need_ohe = ['MONTH', 'DAY_OF_MONTH', 'DAY_OF_WEEK', 'OP_UNIQUE_CARRIER', 'ORIGIN', 'ORIGIN_STATE_FIPS', 'DEST', 'DEP_TIME_BLK', 'ARR_TIME_BLK']

- Weather continuous features that require normalization or standardization

In [ ]:
features_numerical = ['ELEVATION_ORG', 'ELEVATION_DEST', 'SNW_DEST', 'SNW_ORG', 'CIG_ORG', 'WND_ORG', 'GUST_ORG',  'VIS_DEST', 'CIG_DEST', 'WND_DEST', 'GUST_DEST', 'p_rank']

In [ ]:
#features_numerical = ['ELEVATION_ORG', 'ELEVATION_DEST', 'CIG_ORG', 'CIG_DEST', 'p_rank']

In [ ]:
# Null columns
null_cols = ['SLP_ORG', 'VIS_ORG', 'TMP_ORG', 'PREC_ORG', 'DEW_ORG', 'SLP_DEST', 'TMP_DEST', 'DEW_DEST', 'PREC_DEST']

In [ ]:
#null_cols = ['TMP_ORG', 'PREC_ORG', 'DEW_ORG', 'PREC_DEST']

- Weather features that don't require one-hot-encoding (already int 1 or 0)

In [ ]:
wx_categorical = ['TCB_ORG', 'OBSC_ORG', 'FZFG_ORG', 'FZDZ_ORG', 'FZRA_ORG', 'TSRA_ORG', 'HSN_ORG', 'LSN_ORG', 'BSN_ORG', 'VV_ORG', 'TS_ORG', 'HAIL_ORG', 'FOG_ORG', 'VA_ORG', 'BR_ORG', 'SQ_ORG', 'FC_ORG', 'TCB_DEST', 'OBSC_DEST', 'FZFG_DEST', 'FZDZ_DEST', 'FZRA_DEST', 'TSRA_DEST', 'HSN_DEST', 'LSN_DEST', 'BSN_DEST', 'VV_DEST', 'TS_DEST', 'HAIL_DEST', 'FOG_DEST', 'VA_DEST', 'BR_DEST', 'SQ_DEST', 'FC_DEST']

In [ ]:
#wx_categorical = []

### Pipeline Functions

#### Sampling
- Found this function to undersample the majority class
- This step is optional for now
- Source: https://stackoverflow.com/questions/53978683/how-to-undersampling-the-majority-class-using-pyspark

- Example: Passing in a ratio of 2.0 returns a majority class twice as large as the minority class.

In [ ]:
def resample(base_features,ratio,class_field,base_class):
    pos = base_features.filter(col(class_field)==base_class)
    neg = base_features.filter(col(class_field)!=base_class)
    total_pos = pos.count()
    total_neg = neg.count()
    fraction = float(total_pos*ratio)/float(total_neg)
    sampled = neg.sample(False, fraction)
    return sampled.union(pos)

#### Train/Test Split for Small Dataset

In [ ]:
def get_split_small(df):
    # Splitting the data into training and testing, with 80% of the dates in the training set and the remaining 20% in the testing set
    df = df.drop(*drop)
    joined_data_sorted = df.withColumn("rank_p", percent_rank().over(Window.partitionBy().orderBy("FL_DATE")))
    train_df = joined_data_sorted.where("rank_p <= .8").drop("rank_p", "FL_DATE")
    test_df = joined_data_sorted.where("rank_p > .8").drop("rank_p", "FL_DATE")
    return train_df, test_df

#### Train/Test Split for Large Dataset

In [ ]:
def get_split_large(df):
    df = df.drop(*drop)
    train_df = df.where(col('YEAR') != '2018').drop("FL_DATE")
    test_df = df.where(col('YEAR') == '2018').drop("FL_DATE")
    return train_df, test_df

#### Balance Ratio for Logistic Regression

- Creates a new columns called `class_weights` with class percentages that gets passed into the logistic regression as a parameter (`weightCol`)

In [ ]:
def balance_ratio(train_df):
    balance_ratio = train_df.select('DEP_DEL15').filter(col('DEP_DEL15') == 0).count() / train_df.count()
    train_df = train_df.withColumn('class_weights', when(train_df.DEP_DEL15 == 1, balance_ratio).otherwise(1 - balance_ratio))
    return train_df

#### Model Evaluation

In [ ]:
# Evaluate the model on validation set
def model_evaluation(results):
    # Input: prediction results
    
    predictionAndLabels = results.select(['prediction', 'label']\
                                      ).withColumn('label',col('label').cast(DoubleType())).rdd

    metrics = MulticlassMetrics(predictionAndLabels)
    conf_matrix = metrics.confusionMatrix().toArray()
    
    accuracy = (conf_matrix[0][0] + conf_matrix[1][1]) / conf_matrix.sum()
    precision = (conf_matrix[1][1]) / (conf_matrix[0][1] + conf_matrix[1][1])
    recall = (conf_matrix[1][1]) / (conf_matrix[1][0] + conf_matrix[1][1])
    f1 = (2 * precision * recall)/(precision + recall)
    f05 = 1.25 * ((precision * recall) / ((0.25 * precision) + recall))
    f2 = 5 * ((precision * recall) / ((4 * precision) + recall))
        
    print(f'Accuracy = {accuracy:.4f}')
    print(f'Precision = {precision:.4f}')
    print(f'Recall = {recall:.4f}')
    print(f'F1-Score = {f1:.4f}')
    print(f'F05-Score = {f05:.4f}')
    print(f'F2-Score = {f2:.4f}')
    print(f'TP = {conf_matrix[1][1]:.0f}')
    print(f'FN = {conf_matrix[1][0]:.0f}')
    print(f'TN = {conf_matrix[0][0]:.0f}')
    print(f'FP = {conf_matrix[0][1]:.0f}')
    print(f"Total Records: {conf_matrix[0][0] + conf_matrix[1][1] + conf_matrix[0][1] + conf_matrix[1][0]:.0f}")

#### Pipleline Function

In [ ]:
# This pipeline is only used for data preprocessing and the passed model is ignored
def get_pipeline(model, ohe_cols, num_cols, wx_cat, null_col, minmax_std='minmax', pca_k=None):
    
    stages = []
    
    # one-hot-encoding
    stringIndexer = StringIndexer(inputCols=ohe_cols, outputCols=[x + "_Index" for x in ohe_cols], handleInvalid = 'keep')    
    encoder = OneHotEncoder(inputCols=stringIndexer.getOutputCols(), outputCols=[x + "_OHE" for x in need_ohe]) 
    stages += [stringIndexer, encoder]  
    
    # label indexing
    labelToIndex = StringIndexer(inputCol="DEP_DEL15", outputCol="label")
    stages += [labelToIndex]
    
    # Imputation with mean
    imputer = Imputer(strategy='mean', inputCols=null_col, outputCols=[x + "_imp" for x in null_col])
    stages += [imputer]
    
    # Normalization/Standardization   
    if minmax_std == 'minmax':
        scaler_cols = num_cols + [c + "_imp" for c in null_col]
        assembler_1 = VectorAssembler(inputCols=scaler_cols, outputCol='features_a', handleInvalid = 'keep')
        stages += [assembler_1]
        scaler = MinMaxScaler(inputCol='features_a', outputCol='features_scaled', min=0, max=1) 
        stages += [scaler]
    elif minmax_std == 'std':
        scaler_cols = num_cols + [c + "_imp" for c in null_col]
        assembler_1 = VectorAssembler(inputCols=scaler_cols, outputCol='features_a', handleInvalid = 'keep')
        stages += [assembler_1]
        scaler = StandardScaler(inputCol="features_a", outputCol="features_scaled", withStd=True, withMean=False)
        stages += [scaler]
    elif minmax_std == None:
        pass
    else:
        print('Invalid entry in minmax_std parameter')

    # collecting inputs and assemble to one final vector for model
    if minmax_std == None:
        assembler_input = [c + "_Index" for c in ohe_cols] + num_cols + [c + "_imp" for c in null_col] + wx_cat
        assembler_2 = VectorAssembler(inputCols=assembler_input, outputCol="features", handleInvalid = 'keep')
        stages += [assembler_2]
    elif minmax_std == 'minmax' or minmax_std == 'std':
        assembler_input = [c + "_OHE" for c in ohe_cols] + ['features_scaled'] + wx_cat
        assembler_2 = VectorAssembler(inputCols=assembler_input, outputCol="features", handleInvalid = 'keep')
        stages += [assembler_2]
    
    # add pca component
    if pca_k == None:
        pass
    else:
        pca = PCA(k = pca_k, inputCol="features", outputCol="features_pca")
        stages += [pca]
    
    #Not using the passed model as we are going to use our own algorithm from scratch
    #stages += [model]
    
    return Pipeline(stages=stages)

In [ ]:
# This model is not used
model = LogisticRegression(featuresCol="features", labelCol="label", regParam=0.0, maxIter=50)


#### Set up the pipeline

**minmax_std parameter:**
- False = returns pipeline without scaling of continuous features
- 'minmax' = scales continuous features with `MinMaxScaler` (min=0, max=1)
- 'std' = scales continuous features with `StandardScaler` (withStd=True, withMean=False)

In [ ]:
#need_ohe = ['YEAR', 'QUARTER', 'MONTH', 'DAY_OF_MONTH', 'DAY_OF_WEEK', 'OP_UNIQUE_CARRIER', 'ORIGIN', 'ORIGIN_STATE_FIPS', 'DEST', 'DEST_STATE_FIPS', 'DEP_TIME_BLK', 'ARR_TIME_BLK', 'DISTANCE_GROUP', 'BAD_TRAVEL_DAY', 'DEP_TIME_SEGMENT']
# need_ohe = ['QUARTER', 'MONTH', 'DAY_OF_MONTH', 'DAY_OF_WEEK', 'OP_UNIQUE_CARRIER', 'ORIGIN', 'ORIGIN_STATE_FIPS', 'DEST', 'DEST_STATE_FIPS', 'DEP_TIME_BLK', 'ARR_TIME_BLK', 'DISTANCE_GROUP']

#features_numerical = ['ELEVATION_ORG', 'ELEVATION_DEST', 'SNW_DEST', 'SNW_ORG', 'CIG_ORG', 'WND_ORG', 'GUST_ORG',  'VIS_DEST', 'CIG_DEST', 'WND_DEST', 'GUST_DEST', 'p_rank']
# features_numerical = ['ELEVATION_ORG', 'ELEVATION_DEST', 'SNW_DEST', 'SNW_ORG', 'CIG_ORG', 'WND_ORG', 'GUST_ORG',  'VIS_DEST', 'CIG_DEST', 'WND_DEST', 'GUST_DEST']

In [ ]:
pipeline = get_pipeline(model, need_ohe, features_numerical, wx_categorical, null_cols, minmax_std='minmax', pca_k=None)

In [ ]:
pipeline.getStages()

Out[17]: [StringIndexer_ffae98679e6d,
 OneHotEncoder_b38e62ebe5f3,
 StringIndexer_3241056b23e2,
 Imputer_81339dcf99fe,
 VectorAssembler_9b0b62bfe460,
 MinMaxScaler_ba2c7f66e715,
 VectorAssembler_c3b561549795]

#### Get Train and Test Set

In [ ]:
# train/test split on small dataset
train_df, test_df = get_split_small(joined_data_6M)

# train/test split on large (2015-2018) dataset
# train_df, test_df = get_split_large(joined_data_traintest)

# # get holdout (2019) dataset
# holdout = get_holdout(joined_data_holdout)

- Downsample majority class in training set. The second parameter is the class ratio (e.g. 2 would result in a 2:1 majority class ration)

In [ ]:
 train_df = resample(train_df, 1.6, 'DEP_DEL15', 1)

#### Fit the dataset in the pipeline to generate the required features

In [ ]:
pipelineModel = pipeline.fit(train_df)

In [ ]:
train_df = pipelineModel.transform(train_df)

In [ ]:
display(train_df.where(train_df['DAY_OF_MONTH']=='2').limit(2))

YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,OP_UNIQUE_CARRIER,ORIGIN,ORIGIN_STATE_FIPS,DEST,DEST_STATE_FIPS,DEP_DEL15,DEP_TIME_BLK,ARR_TIME_BLK,DISTANCE_GROUP,BAD_TRAVEL_DAY,DEP_TIME_SEGMENT,p_rank,ELEVATION_ORG,SLP_ORG,VIS_ORG,TMP_ORG,DEW_ORG,CIG_ORG,WND_ORG,GUST_ORG,TCB_ORG,OBSC_ORG,PREC_ORG,SNW_ORG,FZFG_ORG,FZDZ_ORG,FZRA_ORG,TSRA_ORG,HSN_ORG,LSN_ORG,BSN_ORG,VV_ORG,TS_ORG,HAIL_ORG,FOG_ORG,VA_ORG,BR_ORG,SQ_ORG,FC_ORG,ELEVATION_DEST,SLP_DEST,VIS_DEST,TMP_DEST,DEW_DEST,CIG_DEST,WND_DEST,GUST_DEST,TCB_DEST,OBSC_DEST,PREC_DEST,SNW_DEST,FZFG_DEST,FZDZ_DEST,FZRA_DEST,TSRA_DEST,HSN_DEST,LSN_DEST,BSN_DEST,VV_DEST,TS_DEST,HAIL_DEST,FOG_DEST,VA_DEST,BR_DEST,SQ_DEST,FC_DEST,YEAR_Index,QUARTER_Index,MONTH_Index,DAY_OF_MONTH_Index,DAY_OF_WEEK_Index,OP_UNIQUE_CARRIER_Index,ORIGIN_Index,ORIGIN_STATE_FIPS_Index,DEST_Index,DEST_STATE_FIPS_Index,DEP_TIME_BLK_Index,ARR_TIME_BLK_Index,DISTANCE_GROUP_Index,BAD_TRAVEL_DAY_Index,DEP_TIME_SEGMENT_Index,YEAR_OHE,QUARTER_OHE,MONTH_OHE,DAY_OF_MONTH_OHE,DAY_OF_WEEK_OHE,OP_UNIQUE_CARRIER_OHE,ORIGIN_OHE,ORIGIN_STATE_FIPS_OHE,DEST_OHE,DEST_STATE_FIPS_OHE,DEP_TIME_BLK_OHE,ARR_TIME_BLK_OHE,DISTANCE_GROUP_OHE,BAD_TRAVEL_DAY_OHE,DEP_TIME_SEGMENT_OHE,label,SLP_ORG_imp,VIS_ORG_imp,TMP_ORG_imp,PREC_ORG_imp,DEW_ORG_imp,SLP_DEST_imp,TMP_DEST_imp,DEW_DEST_imp,PREC_DEST_imp,features_a,features_scaled,features
2015,1,1,2,5,DL,ORD,17,ATL,13,0.0,0600-0659,0900-0959,3,0,2,0.03558293972916778,201.8,1023.0,10.0,-7.20000,-9.40000,72178,6.0,0.0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,307.8,1024.0,10.0,6.70000,5.00000,7998,0.0,0.0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,2.0,16.0,1.0,0.0,1.0,1.0,28.0,12.0,16.0,14.0,0.0,0.0,1.0,"Map(vectorType -> sparse, length -> 1, indices -> List(0), values -> List(1.0))","Map(vectorType -> sparse, length -> 2, indices -> List(0), values -> List(1.0))","Map(vectorType -> sparse, length -> 5, indices -> List(2), values -> List(1.0))","Map(vectorType -> sparse, length -> 31, indices -> List(16), values -> List(1.0))","Map(vectorType -> sparse, length -> 7, indices -> List(1), values -> List(1.0))","Map(vectorType -> sparse, length -> 13, indices -> List(0), values -> List(1.0))","Map(vectorType -> sparse, length -> 2, indices -> List(1), values -> List(1.0))","Map(vectorType -> sparse, length -> 2, indices -> List(1), values -> List(1.0))","Map(vectorType -> sparse, length -> 194, indices -> List(28), values -> List(1.0))","Map(vectorType -> sparse, length -> 51, indices -> List(12), values -> List(1.0))","Map(vectorType -> sparse, length -> 19, indices -> List(16), values -> List(1.0))","Map(vectorType -> sparse, length -> 19, indices -> List(14), values -> List(1.0))","Map(vectorType -> sparse, length -> 10, indices -> List(0), values -> List(1.0))","Map(vectorType -> sparse, length -> 1, indices -> List(0), values -> List(1.0))","Map(vectorType -> sparse, length -> 4, indices -> List(1), values -> List(1.0))",0.0,1023.0,10.0,-7.20000,0.0,-9.40000,1024.0,6.70000,5.00000,0.0,"Map(vectorType -> dense, length -> 21, values -> List(201.8, 307.8, 0.0, 0.0, 72178.0, 6.0, 0.0, 10.0, 7998.0, 0.0, 0.0, 0.03558293972916778, 1023.0, 10.0, -7.2, 0.0, -9.4, 1024.0, 6.7, 5.0, 0.0))","Map(vectorType -> sparse, length -> 21, indices -> List(1, 4, 5, 7, 8, 12, 13, 14, 16, 17, 18, 19), values -> List(0.13069534172050326, 1.0, 0.2006688963210702, 0.125, 0.11080938790213084, 0.6296296296296295, 1.0, 0.2782931354359926, 0.37627811860940685, 0.5694444444444444, 0.5461309523809524, 0.6377295492487479))","Map(vectorType -> sparse, length -> 416, indices -> List(0, 1, 5, 24, 40, 46, 60, 62, 91, 269, 324, 341, 346, 356, 358, 362, 365, 366, 368, 369, 373, 374, 375, 377, 378, 379, 380), values -> List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.13069534172050326, 1.0, 0.2006688963210702, 0.125, 0.11080938790213084, 0.6296296296296295, 1.0, 0.2782931354359926, 0.37627811860940685, 0.5694444444444444, 0.5461309523809524, 0.6377295492487479))"
2015,1,1,2,5,DL,ATL,13,MSP,27,0.0,

In [ ]:
# Select only the features and label
vec_train_df = train_df.select('features','label')

In [ ]:
display(vec_train_df.limit(2))

features,label
"Map(vectorType -> sparse, length -> 416, indices -> List(0, 1, 5, 35, 39, 47, 60, 62, 149, 273, 314, 330, 348, 356, 358, 362, 365, 366, 367, 368, 369, 370, 373, 374, 375, 377, 378, 379, 380), values -> List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.1000085005100306, 1.0, 0.3712374581939799, 0.44345898004434586, 0.125, 1.0, 0.4488188976377953, 0.537037037037037, 1.0, 0.2374768089053803, 0.27402862985685067, 0.4861111111111111, 0.32291666666666663, 0.313856427378965))",0.0
"Map(vectorType -> sparse, length -> 416, indices -> List(0, 1, 5, 35, 39, 52, 59, 61, 67, 261, 321, 334, 346, 356, 358, 361, 362, 365, 368, 369, 370, 372, 373, 374, 375, 377, 378, 379, 380), values -> List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.001147568854131248, 1.0, 0.125, 1.0, 0.2099737532808399, 1.0, 0.7037037037037037, 1.0, 0.463821892393321, 0.5235173824130879, 0.5694444444444444, 0.45535714285714285, 0.41569282136894825))",0.0


In [ ]:
# helper function to parse features and labels
def parse(row):
    """
    Map record_string --> (tuple,of,fields)
    """
    return(row.features.toArray(), int(row.label))

In [ ]:
# Parse the train set features and label into a tuple rdd
vec_train_rdd = vec_train_df.rdd.map(parse).cache()

In [ ]:
# Parse the test set features and label into a tuple rdd
test_df = pipelineModel.transform(test_df)
vec_test_df = test_df.select('features','label')
vec_test_rdd = vec_test_df.rdd.map(parse).cache()

In [ ]:
display(vec_test_df.limit(2))

features,label
"Map(vectorType -> sparse, length -> 416, indices -> List(0, 2, 6, 37, 40, 47, 59, 61, 166, 266, 320, 338, 346, 356, 357, 361, 362, 365, 366, 368, 369, 370, 372, 373, 374, 375, 377, 378, 379, 380, 381), values -> List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.11195171710302619, 0.07491675915649279, 0.17056856187290967, 0.125, 0.048491230014685915, 0.34120734908136485, 1.0, 0.6296296296296295, 1.0, 0.8868274582560297, 0.9325153374233128, 0.4583333333333333, 0.8258928571428571, 0.8964941569282137, 0.012903225806451613))",1.0
"Map(vectorType -> sparse, length -> 416, indices -> List(0, 2, 6, 37, 40, 46, 59, 61, 63, 259, 324, 342, 349, 356, 358, 361, 362, 365, 366, 368, 369, 370, 372, 373, 374, 375, 377, 378, 379, 380, 396), values -> List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0013175790547432847, 0.004189789123196449, 0.2006688963210702, 0.125, 1.0, 0.10761154855643043, 1.0, 0.5740740740740741, 0.1, 0.7829313543599259, 0.936605316973415, 0.5694444444444444, 0.7351190476190476, 0.7495826377295494, 1.0))",0.0


###Logistic Regression Implementation

In [ ]:
# Sigmoid function to be used while calculating the loss
def sigmoid(z):
    return 1.0/(1 + np.exp(-z))

In [ ]:
# Defining log loss/cost function
def LogLoss(dataRDD, W): 
    """
    This function calculates the log loss error
    
    Args:
        dataRDD - with each rdd record as a tuple of (features_array, y)
        W       - (array) model coefficients with bias at index 0
    """
    
    
    # augmenting the features array with a 1 for handling bias at index 0
    augmentedData = dataRDD.map(lambda x: (np.append([1.0], x[0]), x[1]))
    
    # Calculating the loss/cost
    loss = augmentedData.map(lambda x: x[1]*np.log(sigmoid(W.dot(x[0]))) + (1-x[1])*np.log(1 - sigmoid(W.dot(x[0])))).mean()*-1
       
    return loss

In [ ]:
# Defining gradient update function with L1 regularization
def GDUpdate (dataRDD, W, learningRate = 0.15, regParam = 0.1):
    """
    Perform one gradient descent step/update with ridge or lasso regularization.
    Args:
        dataRDD - tuple of (features_array, y)
        W       - (array) model coefficients with bias at index 0
        learningRate - (float) defaults to 0.1 
        regParam - (float) L1 regularization parameter
    Returns:
        model   - (array) updated coefficients, bias at index 0
    """
    # augmented data
    M=dataRDD.count()
    augmentedData = dataRDD.map(lambda x: (np.append([1.0], x[0]), x[1]))
    
    # Broadcast coeeficients
    sc.broadcast(W)  
     
    #Claculate gradient with L1 regularization    
    grad = augmentedData.map(lambda x: ((sigmoid(W.dot(x[0])) - x[1])*x[0])).sum() + regParam * np.append([0.0], np.sign(W)[1:])
    
    # Perform gradient update
    new_model = W - learningRate * grad/M
    
    return new_model

In [ ]:
# Gradient descent function
def GradientDescent(trainRDD, testRDD, wInit, nSteps = 100, learningRate = 0.15, verbose = False):
    """
    Perform nSteps iterations of gradient descent and 
    track loss on a test and train set. Return lists of
    test/train loss and the models themselves.
    """
    # initialize lists to track model performance
    train_history, test_history, model_history = [], [], []
    
    # perform n updates & compute test and train loss after each
    model = wInit
    
    for idx in range(nSteps):  
        # update the model
        model = GDUpdate(trainRDD, model, learningRate)
        
        # keep track of test/train loss for plotting
        train_history.append(LogLoss(trainRDD, model))
        model_history.append(model)
        
        # console output if desired
        if verbose:
            print("----------")
            print(f"STEP: {idx+1}")
            print(f"Model: {[round(w,3) for w in model]}")
            
    # We are not using the test rdd here and the test_history returned will be an empty list
    return train_history, test_history, model_history

In [ ]:
# Initialize the weights based on the number of features + 1
wInit = np.random.uniform(0,1,417)
# Call the gradient descent functon to get the model co-fficients
# Note: even though we are passing the test rdd, we will not be utilizing it in the function
lr_results = GradientDescent(vec_train_rdd, vec_test_rdd, wInit, nSteps = 50, learningRate = 0.21)

In [ ]:
# Check the last three loss results and see loss is not exploding
print(lr_results[0][-3])
print(lr_results[0][-2])
print(lr_results[0][-1])

0.677758447940997
0.6770721608795042
0.6763979369574706

In [ ]:
# Select the last model if the loss was not exploding
model = lr_results[2][-1]

### Predict and Evaluate

In [ ]:
# A function to make predictions using the final model
def makePrediction(testRDD, model, threshold = 0.5):
    """
    Perform predictions on the passed rdd using the model.
    Threshold can be pass to the function to adjust predictions of 1.0 or 0.0
    """
    
    # Augment with 1.0 in the first index to account for bias
    augmentedData = testRDD.map(lambda x: (np.append([1.0], x[0]), x[1]))
    
    # Broadcast model
    sc.broadcast(model)
    
    # Make predictions on the input rdd using the model and the sigmoid function
    predictions = augmentedData.map(lambda x: (float(sigmoid(model.dot(x[0]))), 1.0 if sigmoid(model.dot(x[0])) > threshold else 0.0, x[1]))
    
    return predictions

In [ ]:
model

Out[37]: array([-0.72381087, -1.1246855 , -0.4062588 , -0.21951216, 0.35536067,
 0.25664249, 0.65754238, 0.55563421, 0.57598357, 0.33298641,
 0.12537876, 0.89878517, 0.79454072, 0.18905149, 0.2019363 ,
 0.26696566, 0.5752582 , 0.29941323, 0.6990622 , 0.1293235 ,
 0.12732347, 0.05928256, 0.85774628, 0.27332979, 0.73111292,
 0.49810235, 0.47792647, 0.40679362, 0.15314485, 0.33286552,
 0.54153277, 0.76345517, 0.70615531, 0.55590604, 0.45111347,
 0.83295475, 0.80305077, 0.12306208, -0.01572682, 0.77987038,
 0.55212143, 0.03136683, 0.66197069, 0.44494014, 0.33993856,
 0.4894208 , 0.52412608, 0.11710708, -0.06574068, 0.58980335,
 0.32976991, 0.14810583, 0.71102577, 0.10273516, 0.21722007,
 0.6749819 , 0.42509662, 0.23753319, 0.19196214, 0.43325739,
 -0.12893252, -0.14826126, 0.08252554, -0.06650305, 0.95013233,
 0.6288149 , 0.51164933, 0.16460628, 0.89940596, 0.4978565 ,
 0.2206387 , 0.7587468 , 0.66383352, 0.09154631, 0.062548 ,
 0.75977073, 0.34027121, 0.78347201, 0.47699406, 0.75335788,
 0.44692453, 0.61690107, 0.23633522, 0.5893679 , 0.60747081,
 0.25107948, 0.47055108, 0.39070081, 0.02621635, 0.82351273,
 0.77839993, 0.18726527, 0.01334593, 0.13964383, 0.77354771,
 0.05612801, 0.97959118, 0.4724752 , 0.22314416, 0.72189044,
 0.13725318, 0.23492056, 0.86064825, 0.53149547, 0.04121841,
 0.28961128, 0.23978599, 0.56803415, 0.92751843, 0.58868391,
 0.15274404, 0.10503604, 0.74269302, 0.51145139, 0.53761923,
 0.76122116, 0.02470783, 0.39027979, 0.8784624 , 0.695972 ,
 0.58821421, 0.09318459, 0.06718856, 0.09162267, 0.87649944,
 0.76608982, 0.34752002, 0.64151904, 0.59531182, 0.65683544,
 0.79120329, 0.73047662, 0.30550814, 0.85711659, 0.57002591,
 0.33719192, 0.7407694 , 0.34416579, 0.10973908, 0.83047871,
 0.94484227, 0.70509734, 0.79825329, 0.05029169, 0.78189364,
 0.52647978, 0.83434847, 0.58551342, 0.89441697, 0.34955371,
 0.43620163, 0.62763602, 0.4092811 , 0.20944147, 0.9677799 ,
 0.04261095, 0.01305732, 0.81113285, 0.68745248, 0.81796168,
 0.76279329, 0.34472286, 0.78708806, 0.99592196, 0.55987739,
 0.85107968, 0.35455833, 0.45381992, 0.10234107, 0.81127434,
 0.10435803, 0.57173738, 0.53794134, 0.26469128, 0.63948482,
 0.06232467, 0.27147021, 0.22287543, 0.11019345, 0.86543531,
 0.38461873, 0.68419617, 0.67856825, 0.64003004, 0.09317761,
 0.58257297, 0.52943654, 0.28536579, 0.74323901, 0.10009226,
 0.541696 , 0.59300769, 0.09023225, 0.46701684, 0.74424991,
 0.88040149, 0.51547189, 0.6956731 , 0.41335087, 0.68880674,
 0.29993176, 0.82753618, 0.76489906, 0.20888959, 0.99258805,
 0.09050906, 0.6457897 , 0.70877516, 0.34266413, 0.75464869,
 0.90355693, 0.92915417, 0.25878756, 0.30142649, 0.6049528 ,
 0.03827015, 0.8353975 , 0.04820384, 0.34445118, 0.62883064,
 0.91060769, 0.30291082, 0.1501071 , 0.12836628, 0.1013758 ,
 0.48919515, 0.581361 , 0.09618723, 0.34965499, 0.56914093,
 0.13229885, 0.57747329, 0.64724078, 0.36107009, 0.74067887,
 0.12613862, 0.79868864, 0.74870748, 0.75502629, 0.60485278,
 0.2217843 , 0.370628 , 0.74013252, 0.64897019, 0.59046786,
 0.15473947, 0.35439628, 0.03350544, 0.00487193, 0.22912465,
 0.99905517, 0.60975121, 0.17409965, 0.76560943, 0.36404469,
 0.64803702, 0.12668567, 0.34688751, 0.34337032, 0.49791254,
 0.4061433 , 0.48126335, 0.00449537, 0.46973531, 0.42376347,
 0.30302369, 0.36254601, 0.68512885, -0.00950904, 0.30277533,
 0.78398518, 0.96700434, 0.45616792, 0.54854944, 0.93706702,
 0.67508294, 0.89209705, 0.71826331, 0.61639144, 0.86163474,
 0.61152663, 0.36472866, 0.19451811, 0.50694143, 0.08619562,
 0.18481933, 0.63994954, 0.24291212, 0.31191696, 0.43910288,
 0.72485129, 0.65940724, 0.70280968, 0.33898416, 0.6119606 ,
 0.32765437, 0.9594578 , 0.10053936, 0.97318091, 0.76051578,
 0.44437976, 0.92704427, 0.8852614 , 0.03046377, 0.6776801 ,
 0.29076463, 0.45377106, 0.6949497 , 0.94368793, 0.01233303,
 0.19574412, 0.58776103, 0.16851774, 0.49222228, 0.03115405,
 0.69205654, 0.44296537, 0.42097493, 0.67642354, 0.31493199,
 0.26642891, 0.69320997, 0.0467741 , 0.39460468, 0.62497021,
 0.45106854, 0.

In [ ]:
# Make predictions on the test rdd
predrdd = makePrediction(vec_test_rdd, model,threshold = 0.35)

In [ ]:
# Define the predicted resuts schema
schema = StructType([
StructField("prob", FloatType(), True),
StructField("prediction", DoubleType(), True),
StructField("label", IntegerType(), True)])

In [ ]:
# Convert the predicted results to a dataframe
preddf = sqlContext.createDataFrame(predrdd, schema)

In [ ]:
display(preddf.limit(2))

prob,prediction,label
0.146727,0.0,1
0.28866726,0.0,0


###Metrics

In [ ]:
# Evaluate metrics on the predcted results dataframe
model_evaluation(preddf)

Accuracy = 0.6875
Precision = 0.3797
Recall = 0.2208
F1-Score = 0.2792
F05-Score = 0.3319
F2-Score = 0.2409
TP = 3895
FN = 13748
TN = 40346
FP = 6363
Total Records: 64352

### Training on a portion of Large dataset

In [ ]:
def get_split_large_par(df):
    df = df.drop(*drop)
    train_df = df.where((col('YEAR') == '2018') & (col('MONTH') == '2')).drop("FL_DATE")
    test_df = df.where((col('YEAR') == '2018') & (col('MONTH') == '3')).drop("FL_DATE")
    return train_df, test_df

In [ ]:
# Create train and test data and resample
train_df, test_df = get_split_large_par(joined_data_traintest)
train_df = resample(train_df, 1.7, 'DEP_DEL15', 1)

In [ ]:
# Perform data processing using the pipeline and fit train data
pipelineModel = pipeline.fit(train_df)
train_df = pipelineModel.transform(train_df)
vec_train_df = train_df.select('features','label')
vec_train_rdd = vec_train_df.rdd.map(parse).cache()

# Transform test data using fitted train data
test_df = pipelineModel.transform(test_df)
vec_test_df = test_df.select('features','label')
vec_test_rdd = vec_test_df.rdd.map(parse).cache()

In [ ]:
display(vec_train_df.take(2))

features,label
"Map(vectorType -> sparse, length -> 939, indices -> List(0, 1, 2, 17, 33, 48, 101, 403, 456, 792, 830, 852, 869, 879, 880, 884, 885, 888, 889, 891, 892, 893, 895, 896, 897, 898, 900, 901, 902, 903), values -> List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.15572934376062564, 0.11284427065623939, 1.0, 0.25984251968503935, 0.22222222222222224, 1.0, 0.2222222222222222, 0.12332561516426588, 0.6615384615384616, 0.22222222222222224, 0.6720827178729691, 0.47649918962722854, 0.5396825396825397, 0.5809379727685325, 0.4903536977491961))",0.0
"Map(vectorType -> sparse, length -> 939, indices -> List(0, 1, 2, 28, 37, 49, 58, 391, 629, 779, 843, 858, 868, 879, 881, 884, 885, 888, 889, 891, 892, 893, 895, 896, 897, 898, 900, 901, 902, 903, 919, 936), values -> List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.07242434546072764, 0.0029751785107106432, 0.047078057025686494, 0.29133858267716534, 0.11111111111111112, 0.0041286818698218295, 0.3611111111111111, 0.831163441543136, 0.5538461538461539, 0.11111111111111112, 0.6070901033973414, 0.6564019448946515, 0.5238095238095237, 0.8245083207261725, 0.9196141479099679, 1.0, 1.0))",0.0


In [ ]:
display(vec_test_df.take(2))

features,label
"Map(vectorType -> sparse, length -> 939, indices -> List(0, 1, 16, 33, 53, 173, 393, 724, 798, 838, 864, 870, 879, 881, 884, 885, 888, 889, 891, 892, 893, 894, 895, 896, 897, 898, 900, 901, 902, 903), values -> List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0031026861611696704, 0.09626827609656581, 1.0, 0.1837270341207349, 0.22222222222222224, 1.0, 0.275, 0.33125, 0.013377338712258793, 0.4307692307692308, 0.22222222222222224, 0.8360413589364846, 0.9092382495948136, 0.5873015873015872, 0.47806354009077157, 0.4678456591639871))",0.0
"Map(vectorType -> sparse, length -> 939, indices -> List(0, 1, 7, 31, 41, 68, 404, 570, 819, 841, 855, 871, 879, 880, 884, 885, 888, 889, 891, 892, 893, 895, 896, 897, 898, 900, 901, 902, 903), values -> List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.11284427065623939, 0.11645698741924516, 0.3463659286763279, 0.31758530183727035, 0.22222222222222224, 1.0, 0.3361111111111111, 0.6783038531349699, 0.7230769230769232, 0.22222222222222224, 0.5657311669128509, 0.3954619124797407, 0.6666666666666666, 0.5128593040847201, 0.527331189710611))",0.0


In [ ]:
# Initialize model with number of features + 1 for bias
wInit = np.random.uniform(0,1,940)
# Perform gradient descent on the train set
lr_results = GradientDescent(vec_train_rdd, vec_test_rdd, wInit, nSteps = 50, learningRate = 0.28)

In [ ]:
# Check the last three loss results and see loss is not exploding
# The above results are valid if the loss was not exploding
print(lr_results[0][-3])
print(lr_results[0][-2])
print(lr_results[0][-1])

0.7090909255515747
0.707826963126258
0.7065900531797258

In [ ]:
# Pick the last model
model = lr_results[2][-1]

In [ ]:
# Make predictions
predrdd = makePrediction(vec_test_rdd, model,threshold = 0.40)
preddf = sqlContext.createDataFrame(predrdd, schema)
# Evaluate results
model_evaluation(preddf)

Accuracy = 0.2711
Precision = 0.1697
Recall = 0.8744
F1-Score = 0.2842
F05-Score = 0.2023
F2-Score = 0.4776
TP = 85929
FN = 12347
TN = 75106
FP = 420542
Total Records: 593924

###Testing on the full dataset
Training on 2015 to 2017 records<br/>
Testing on 2018 records

In [ ]:
def get_split_large_par(df):
    df = df.drop(*drop)
    train_df = df.where(col('YEAR') != '2018').drop("FL_DATE")
    test_df = df.where(col('YEAR') == '2018').drop("FL_DATE")
    return train_df, test_df

In [ ]:
# 2015 to 2017 data for training and 2018 data for testing
train_df_l, test_df_l = get_split_large_par(joined_data_traintest)

In [ ]:
# Undersample the majority class
train_df_l = resample(train_df_l, 1.7, 'DEP_DEL15', 1)

In [ ]:
# Fit train dataset for feature processing
pipelineModel = pipeline.fit(train_df_l)
train_df_l = pipelineModel.transform(train_df_l)
vec_train_df_l = train_df_l.select('features','label')
vec_train_rdd_l = vec_train_df_l.rdd.map(parse).cache()

# Transfor the test dataset
test_df_l = pipelineModel.transform(test_df_l)
vec_test_df_l = test_df_l.select('features','label')
vec_test_rdd_l = vec_test_df_l.rdd.map(parse).cache()

In [ ]:
display(vec_train_df_l.take(2))

features,label
"Map(vectorType -> sparse, length -> 948, indices -> List(1, 3, 9, 37, 54, 60, 72, 407, 649, 801, 852, 871, 877, 887, 891, 893, 894, 897, 898, 900, 901, 904, 905, 906, 907, 909, 910, 911, 912), values -> List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.08564263855831351, 0.12036722203332201, 0.12467787968633101, 0.12359550561797754, 0.1098901098901099, 1.0, 0.8233509086568446, 0.5567010309278351, 0.125, 0.7663157894736842, 0.7982456140350876, 0.5656565656565657, 0.7447698744769876, 0.8338235294117647))",0.0
"Map(vectorType -> sparse, length -> 948, indices -> List(2, 6, 14, 42, 51, 62, 84, 418, 649, 801, 853, 868, 879, 887, 891, 893, 894, 897, 900, 901, 904, 905, 906, 907, 909, 910, 911, 912), values -> List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 7.650459027541654E-4, 0.12036722203332201, 1.0, 0.1098901098901099, 0.13854637147053117, 0.4113487722258379, 0.5463917525773195, 0.125, 0.5084210526315789, 0.4809941520467836, 0.5454545454545455, 0.45920502092050214, 0.48235294117647054))",0.0


In [ ]:
display(vec_test_df_l.take(2))

features,label
"Map(vectorType -> sparse, length -> 948, indices -> List(5, 15, 33, 50, 86, 413, 649, 801, 852, 858, 877, 887, 891, 893, 894, 897, 900, 901, 902, 904, 905, 906, 907, 909, 910, 911, 912), values -> List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.09418565113906835, 0.12036722203332201, 1.0, 0.1098901098901099, 0.05126215744409653, 0.036204744069912614, 0.3230417664908438, 0.5670103092783505, 0.125, 0.8073684210526316, 0.8713450292397661, 0.595959595959596, 0.6631799163179918, 0.7602941176470589))",0.0
"Map(vectorType -> sparse, length -> 948, indices -> List(6, 14, 25, 52, 86, 413, 649, 801, 851, 867, 877, 887, 890, 893, 894, 897, 898, 900, 901, 902, 904, 905, 906, 907, 909, 910, 911, 912), values -> List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.09418565113906835, 0.12036722203332201, 1.0, 0.06367041198501873, 0.1098901098901099, 1.0, 0.08739076154806492, 0.3230417664908438, 0.7525773195876289, 0.125, 0.3915789473684211, 0.38304093567251457, 0.7777777777777778, 0.3378661087866109, 0.3191176470588235))",1.0


In [ ]:
# Initialize model
wInit = np.random.uniform(0,1,949)
# Perform gradient descent
lr_results_l = GradientDescent(vec_train_rdd_l, vec_test_rdd_l, wInit, nSteps = 125, learningRate = 0.28)
# Pick the last model (the model will be checked if the loss did not explode in next step)
model_l = lr_results_l[2][-1]
#make predictions
predrdd_l = makePrediction(vec_test_rdd_l, model_l,threshold = 0.38)
preddf_l = sqlContext.createDataFrame(predrdd_l, schema)
# Evaluate resuts
model_evaluation(preddf_l)

Accuracy = 0.5995
Precision = 0.2368
Recall = 0.5288
F1-Score = 0.6435
F05-Score = 0.2662
F2-Score = 0.4242
TP = 690272
FN = 614982
TN = 3560107
FP = 2225087
Total Records: 7090448

In [ ]:
# Check the last three loss results and see loss is not exploding
# The above results are valid if the loss was not exploding
#lr_results_l[0][-3]
#lr_results_l[0][-2]
lr_results_l[0][-1]

Out[256]: 0.6666474051298822

In [ ]:
# Make another set of predictions by lowering the threshold value
predrdd_l = makePrediction(vec_test_rdd_l, model_l,threshold = 0.25)

In [ ]:
# Convert predictions to dataframe
preddf_l = sqlContext.createDataFrame(predrdd_l, schema)

In [ ]:
# Evaluate predictions
model_evaluation(preddf_l)

Accuracy = 0.4309
Precision = 0.2133
Recall = 0.7778
F1-Score = 0.4718
F05-Score = 0.2495
F2-Score = 0.5086
TP = 1015262
FN = 289992
TN = 2039996
FP = 3745198
Total Records: 7090448